In [1]:
%matplotlib inline


Sequence Models and Long-Short Term Memory Networks
===================================================

At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding *hidden state* $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.


**LSTM's in Pytorch**


Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence
import datetime

torch.manual_seed(1)

In [6]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
inputs

[tensor([[ 0.5119, -0.6368,  1.0429]]),
 tensor([[ 0.4903,  1.0318, -0.5989]]),
 tensor([[ 1.6015, -1.0735, -1.2173]]),
 tensor([[ 0.6472, -0.0412, -0.1775]]),
 tensor([[-0.5000,  0.8673, -0.2732]])]

In [7]:
torch.cat(inputs).view(len(inputs), 1, -1)

tensor([[[ 0.5119, -0.6368,  1.0429]],

        [[ 0.4903,  1.0318, -0.5989]],

        [[ 1.6015, -1.0735, -1.2173]],

        [[ 0.6472, -0.0412, -0.1775]],

        [[-0.5000,  0.8673, -0.2732]]])

In [4]:
# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
hidden

(tensor([[[ 0.5438, -0.4057,  1.1341]]]),
 tensor([[[-1.1115,  0.3501, -0.7703]]]))

In [5]:
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


**Example: An LSTM for Part-of-Speech Tagging**

In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.


Prepare data:



In [11]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [12]:
training_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']),
 (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]

Create the model:



In [13]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [14]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.1737, -1.1350, -0.9960],
        [-1.1711, -1.2007, -0.9442],
        [-1.2051, -1.1736, -0.9389],
        [-1.1577, -1.1514, -0.9954],
        [-1.1466, -1.1302, -1.0236]])
tensor([[-0.0930, -2.8627, -3.4529],
        [-2.7217, -0.1597, -2.5029],
        [-2.6742, -3.9109, -0.0932],
        [-0.1738, -3.2647, -2.1095],
        [-4.5002, -0.0148, -5.6273]])


**Exercise: Augmenting the LSTM part-of-speech tagger with character-level features**

In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.




In [20]:
import string
string.printable + 'ğüşıöçÜŞİÖÇ'

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0cğüşıöçÜŞİÖÇ'

In [37]:
char_to_ix = {char:enum for (enum,char) in enumerate(string.printable)}
char_to_ix['<PAD>'] = len(char_to_ix)
print(char_to_ix['\''])

CHAR_EMBEDDING_DIM = 3

def prepare_char_sequence(seq):
    seq = [prepare_sequence(w, char_to_ix) for w in seq]
    max_len = max([len(w) for w in seq])
    seq = [torch.cat((w, torch.tensor([char_to_ix['<PAD>']]*(max_len-len(w)),dtype=torch.long))) for w in seq]
    return seq

68


In [23]:
training_data[0][0]

['The', 'dog', 'ate', 'the', 'apple']

In [22]:
[prepare_sequence(w, char_to_ix) for w in training_data[0][0]]

[tensor([55, 17, 14]),
 tensor([13, 24, 16]),
 tensor([10, 29, 14]),
 tensor([29, 17, 14]),
 tensor([10, 25, 25, 21, 14])]

In [29]:
max_len=5
w = 'dog'
torch.cat((torch.tensor([55, 17, 14]), torch.tensor([-1]*(max_len-len(w)))))

tensor([55, 17, 14, -1, -1])

In [99]:
class CharEmbLSTMTagger(nn.Module):

    def __init__(self, embedding_dim, char_embedding_dim, hidden_dim, charset_size, vocab_size, tagset_size, pad):
        super(CharEmbLSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.char_embeddings = nn.Embedding(charset_size, char_embedding_dim, padding_idx=pad)
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm_char = nn.LSTM(char_embedding_dim, char_embedding_dim)
        self.lstm_word = nn.LSTM(embedding_dim + char_embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, char_sentence, sentence):
        word_char_embeds = []
        for i in range(len(char_sentence)):
            char_embeds = self.char_embeddings(char_sentence[i])
            char_lstm_out, (hidden, cell) = self.lstm_char(char_embeds.view(len(char_sentence[i]),1,-1))
            word_embeds = self.word_embeddings(sentence[i])
            word_char_embeds.append(torch.cat((word_embeds, hidden.view(-1)), dim=0))

        word_char_embeds = torch.stack(word_char_embeds)
        lstm_out, _ = self.lstm_word(word_char_embeds.view(len(sentence), 1, -1))

        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [100]:
t0 = datetime.datetime.now()
model = CharEmbLSTMTagger(EMBEDDING_DIM, CHAR_EMBEDDING_DIM, HIDDEN_DIM, len(char_to_ix), len(word_to_ix), len(tag_to_ix), 
                          char_to_ix['<PAD>'])
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    char_inputs = prepare_char_sequence(training_data[0][0])
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(char_inputs, inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        char_sentence_in = prepare_char_sequence(sentence)
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(char_sentence_in, sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    char_inputs = prepare_char_sequence(training_data[0][0])
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(char_inputs, inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)
t1 = datetime.datetime.now()
print('Total Time:', t1-t0)

tensor([[-1.3538, -1.0000, -0.9838],
        [-1.4256, -0.9397, -0.9973],
        [-1.3298, -1.0734, -0.9323],
        [-1.3747, -1.0070, -0.9629],
        [-1.3811, -0.9648, -1.0006]])
tensor([[-0.0651, -3.8197, -3.1920],
        [-3.5934, -0.0310, -5.8072],
        [-3.0541, -4.5455, -0.0595],
        [-0.0404, -3.6910, -4.2219],
        [-4.1262, -0.0185, -6.1418]])
Total Time: 0:00:08.738854


In [91]:
import datetime

In [40]:
#w[0]
torch.tensor([[-0.1778, -3.0725, -2.1494],
        [-2.7127, -0.0742, -5.2585],
        [-1.7926, -3.1809, -0.2333],
        [-0.1475, -3.3955, -2.2672],
        [-3.9127, -0.0239, -5.6226]]).argmax(dim=1)

tensor([0, 1, 2, 0, 1])

In [41]:
#ilk, örnekteki, char_embedding eklenmemiş hali
torch.tensor([[-0.0930, -2.8627, -3.4529], 
        [-2.7217, -0.1597, -2.5029],
        [-2.6742, -3.9109, -0.0932],
        [-0.1738, -3.2647, -2.1095],
        [-4.5002, -0.0148, -5.6273]]).argmax(dim=1)

tensor([0, 1, 2, 0, 1])

In [45]:
#w[1]
torch.tensor([[-0.0162, -5.4440, -4.4425],
        [-5.5069, -0.0136, -4.6586],
        [-4.1085, -4.2194, -0.0316],
        [-0.0169, -4.4673, -5.2465],
        [-5.9041, -0.0062, -5.6639]]).argmax(dim=1)

tensor([0, 1, 2, 0, 1])

In [59]:
sentence =       ['benim', 'adim', 'ne', '?']
sentence =       [0      , 1     , 2   , 3  ]
char_sentence =  [['b','e','n','i','m'],['a','d','i','m'],['n','e'],['?']]
char_sentence =  [[0  ,1  ,2  ,3  ,4  ],[5  ,6  ,3  ,4  ],[2  ,1  ],[7]]

In [101]:
def prepare_char_sequence(seq):
    seq =  [[char_to_ix[c] for c in w]for w in seq]
    lengths = torch.LongTensor(list(map(len, seq)))
    seq_tensor = Variable(torch.zeros((len(seq), lengths.max()))).long()
    # seq_tensor => [[0 0 0 0 0 0 0 0]
    #                [0 0 0 0 0 0 0 0]
    #                [0 0 0 0 0 0 0 0]]

    for idx, (s, seqlen) in enumerate(zip(seq, lengths)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(s)
    # seq_tensor => [[ 6  9  8  4  1 11 12 10]          # long_str
    #                [12  5  8 14  0  0  0  0]          # tiny
    #                [ 7  3  2  5 13  7  0  0]]         # medium
    # seq_tensor.shape : (batch_size X max_seq_len) = (3 X 8)
    lengths, perm_idx = lengths.sort(0, descending=True)
    seq = seq_tensor[perm_idx]
    # seq = seq.transpose(0, 1)
    return seq, lengths

In [102]:
class CharEmbLSTMTaggerPacked(nn.Module):

    def __init__(self, embedding_dim, char_embedding_dim, hidden_dim, charset_size, vocab_size, tagset_size):
        super(CharEmbLSTMTaggerPacked, self).__init__()
        self.hidden_dim = hidden_dim

        self.char_embeddings = nn.Embedding(charset_size, char_embedding_dim)
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm_char = nn.LSTM(char_embedding_dim, char_embedding_dim, batch_first=True)
        self.lstm_word = nn.LSTM(embedding_dim + char_embedding_dim, hidden_dim, batch_first=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, char_sentence, char_lengths, sentence):

        char_embeds = self.char_embeddings(char_sentence)
        packed = pack_padded_sequence(char_embeds, char_lengths, batch_first=True)
        packed_out, (hidden, cell) = self.lstm_char(packed)

        word_embeds = self.word_embeddings(sentence)
        word_char_embeds = torch.cat((word_embeds, hidden.view(len(sentence), -1)), dim=1)

        # word_char_embeds = torch.stack(word_char_embeds)
        lstm_out, _ = self.lstm_word(word_char_embeds.view(len(sentence), 1, -1))

        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores


In [108]:
t0 = datetime.datetime.now()
model = CharEmbLSTMTaggerPacked(EMBEDDING_DIM, CHAR_EMBEDDING_DIM, HIDDEN_DIM, len(char_to_ix), len(word_to_ix), 
                                len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    char_inputs, char_lengths = prepare_char_sequence(training_data[0][0])
    sentence_in = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(char_inputs, char_lengths, sentence_in)
    print(tag_scores)

for epoch in range(500):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        char_sentence_in, char_lengths = prepare_char_sequence(sentence)
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(char_sentence_in, char_lengths, sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    char_inputs, char_lengths = prepare_char_sequence(training_data[0][0])
    sentence_in = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(char_inputs, char_lengths, sentence_in)

   # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)
t1 = datetime.datetime.now()
print('Total Time:', t1-t0)

tensor([[-1.5065, -0.8503, -1.0468],
        [-1.3096, -1.0353, -0.9810],
        [-1.5137, -0.8434, -1.0509],
        [-1.5595, -0.8359, -1.0321],
        [-1.4699, -0.9084, -1.0027]])
tensor([[-0.0102, -4.8130, -6.2141],
        [-4.9879, -0.0114, -5.3988],
        [-5.3744, -3.9994, -0.0232],
        [-0.0125, -4.6764, -5.7884],
        [-5.7297, -0.0070, -5.6032]])
Total Time: 0:00:03.186864


In [ ]:
enforce_sorted=True

In [82]:
torch.nn.utils.rnn.pack_padded_sequence(inp,[5,4,221], batch_first=True,enforce_sorted=True)

TypeError: pack_padded_sequence() got an unexpected keyword argument 'enforce_sorted'

In [ ]:
class SequenceWise(nn.Module):
    def __init__(self, module):
        """
        Collapses input of dim T*N*H to (T*N)*H, and applies to a module.
        Allows handling of variable sequence lengths and minibatch sizes.
        :param module: Module to apply input to.
        """
        super(SequenceWise, self).__init__()
        self.module = module

    def forward(self, x):
        t, n = x.size(0), x.size(1)
        x = x.view(t * n, -1)
        x = self.module(x)
        x = x.view(t, n, -1)
        return x

    def __repr__(self):
        tmpstr = self.__class__.__name__ + ' (\n'
        tmpstr += self.module.__repr__()
        tmpstr += ')'
        return tmpstr
    
class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        t, n = x.size(0), x.size(1) 
        # merge batch and seq dimensions
        x_reshape = x.contiguous().view(t * n, x.size(2))
        y = self.module(x_reshape)
        # We have to reshape Y
        y = y.contiguous().view(t, n, y.size()[1])
        return y
    
# you can do TimeDistributed in pytorch with a regular for loop.


# packing might speed up; but no ggod for now cause fucks up the out and hidden... or the current shit is fucked
packed = pack_padded_sequence(x, lengths, batch_first=True)

# Forward propagate RNN
out, _ = self.rnn(packed)

# Reshape *final* output to (batch_size, hidden_size)
padded = pad_packed_sequence(out, batch_first=True)

``Right, you don’t have to use pack_padded_sequence. Padding is fine, but it is different from using pack_padded_seq. For packed input, RNN will not perform calculation on pad elements.

For example, you have a padded mini batch (size 2), zero is padding.

1 1 1

1 0 0

The output will be 3 (seq length) x 2 (batch size). However, packed input will result in a packed output contains (3 x 1 and 1 x 1). If you feed pack into RNN, it will not calculate output for your pad element. Moreover, hidden will be the hidden after the last valid input instead of hidden after the last zero padding (if you feed pad into rnn, hidden will be in the case).

RNN actually does not distinguish pad and valid elements, and it performs the same calculation on them. You may need to clean the output (e.g., mask output) to get the result you want. Dynamic RNN (feed with packed input) does not have this problem.
``

https://discuss.pytorch.org/t/understanding-pack-padded-sequence-and-pad-packed-sequence/4099/4

https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e

https://gist.github.com/HarshTrivedi/f4e7293e941b17d19058f6fb90ab0fec

https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch

I have stumbled upon this problem too and below is what I figured out.

When training RNN (LSTM or GRU or vanilla-RNN), it is difficult to batch the variable length sequences. For ex: if length of sequences in a size 8 batch is [4,6,8,5,4,3,7,8], you will pad all the sequences and that will results in 8 sequences of length 8. You would end up doing 64 computation (8x8), but you needed to do only 45 computations. Moreover, if you wanted to do something fancy like using a bidirectional-RNN it would be harder to do batch computations just by padding and you might end up doing more computations than required.

Instead, pytorch allows us to pack the sequence, internally packed sequence is a tuple of two lists. One contains the elements of sequences. Elements are interleaved by time steps (see example below) and other contains the size of each sequence the batch size at each step. This is helpful in recovering the actual sequences as well as telling RNN what is the batch size at each time step. This has been pointed by @Aerin. This can be passed to RNN and it will internally optimize the computations.

I might have been unclear at some points, so let me know and I can add more explanations. 